#### Download the Dataset
We use Kaggle to download the RAF-DB facial emotion dataset, which contains images labeled with different emotions. This dataset is needed for training our model.

In [18]:
!kaggle datasets download -d shuvoalok/raf-db-dataset

Dataset URL: https://www.kaggle.com/datasets/shuvoalok/raf-db-dataset
License(s): other
  0% 0.00/37.7M [00:00<?, ?B/s]
100% 37.7M/37.7M [00:00<00:00, 1.31GB/s]


#### Import Image Data Generator
We import a tool from TensorFlow called `ImageDataGenerator`. This helps us load images and apply simple transformations (like resizing or rotating) to make our model more robust.

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#### Set Paths for Training and Testing Data
We specify where our training and testing images are located. These paths tell our code where to find the images after downloading and extracting the dataset.

In [19]:
train_path = "/content/archive/archive/train"
test_path = "/content/archive/archive/test"

#### Prepare Image Data Generators
We create two data generators:
- One for training images, which also applies random changes (like rotation, shifting, zooming, brightness, and flipping) to help the model learn better.
- One for testing images, which only rescales the pixel values.

In [20]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.05,
    brightness_range=[0.9, 1.1],
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(
    rescale=1./255
)

#### Load Images from Folders
We use the data generators to load images from the specified folders. The images are resized to 100x100 pixels, and the labels are set up for multi-class classification (since we have multiple emotions).

In [21]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=32
)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=32
)

Found 12271 images belonging to 7 classes.
Found 3068 images belonging to 7 classes.


#### Import Model Building Tools
We import necessary components from TensorFlow and Keras to build our neural network. These include layers for convolution, pooling, flattening, normalization, and dropout.

In [22]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Conv2D,Dense,MaxPool2D,Flatten,BatchNormalization,Dropout

#### Build the Convolutional Neural Network (CNN)
We create a CNN model. This type of model is good at recognizing patterns in images. The model has several layers:
- Convolutional layers to detect features
- Pooling layers to reduce size
- Dense layers to make predictions
- Batch normalization to stabilize learning
- The final layer predicts one of seven emotions

In [23]:
model = Sequential()

model.add(Conv2D(64,kernel_size=(3,3),activation='relu',input_shape=(100,100,3)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(7,activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### Show Model Summary
We display a summary of our model, showing the layers and the number of parameters in each layer.

In [24]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 98, 98, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 96, 96, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 46, 46, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 44, 44, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 30976)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,965,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,096,071 (15.63 MB)

 Trainable params: 4,095,559 (15.62 MB)

 Non-trainable params: 512 (2.00 KB)

#### Compile the Model
We set up the model for training by choosing an optimizer (rmsprop), a loss function (categorical crossentropy), and a metric (accuracy) to track performance.

In [25]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

#### Train the Model
We train the model using our training images for 100 epochs (cycles through the data). We also check how well the model performs on the test images after each epoch.

In [26]:
model.fit(train_generator,epochs=100,validation_data=test_generator)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
384/384 ━━━━━━━━━━━━━━━━━━━━ 62s 142ms/step - accuracy: 0.3880 - loss: 1.7085 - val_accuracy: 0.5476 - val_loss: 1.3896
Epoch 2/100
384/384 ━━━━━━━━━━━━━━━━━━━━ 72s 131ms/step - accuracy: 0.5373 - loss: 1.2752 - val_accuracy: 0.6177 - val_loss: 1.0547
Epoch 3/100
384/384 ━━━━━━━━━━━━━━━━━━━━ 49s 128ms/step - accuracy: 0.5953 - loss: 1.1137 - val_accuracy: 0.6115 - val_loss: 1.1294
Epoch 4/100
384/384 ━━━━━━━━━━━━━━━━━━━━ 50s 130ms/step - accuracy: 0.6410 - loss: 0.9977 - val_accuracy: 0.6838 - val_loss: 0.9684
Epoch 5/100
384/384 ━━━━━━━━━━━━━━━━━━━━ 82s 129ms/step - accuracy: 0.6694 - loss: 0.9308 - val_accuracy: 0.7093 - val_loss: 0.8050
Epoch 6/100
384/384 ━━━━━━━━━━━━━━━━━━━━ 50s 129ms/step - accuracy: 0.6828 - loss: 0.8792 - val_accuracy: 0.7066 - val_loss: 0.8197
Epoch 7/100
384/384 ━━━━━━━━━━━━━━━━━━━━ 49s 129ms/step - accuracy: 0.6999 - loss: 0.8288 - val_accuracy: 0.6763 - val_loss: 0.9405
Epoch 8/100
384/384 ━━━━━━━━━━━━━━━━━━━━ 48s 125ms/step - accuracy: 0.7112 -

#### Save the Trained Model
We save our trained model to a file so we can use it later without retraining.

In [28]:
from keras.models import save_model
save_model(model,'cnn.keras')